## This is a mockup of the machine learning model we consider using for our project. ##
### We have encoded the dateset and scaled it, but the data has not been cleaned or otherwise preprocessed yet. 


In [3]:
# Initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [4]:
# Loading data
file_path = Path("../Resources/Travel.csv")
df = pd.read_csv(file_path)
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [31]:
# Generate our categorical variable list
df_cat = df.dtypes[df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
df[df_cat].nunique()

TypeofContact     2
Occupation        4
Gender            3
ProductPitched    5
MaritalStatus     4
Designation       5
dtype: int64

In [32]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[df_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(df_cat)
encode_df.head()

,TypeofContact_Company Invited,TypeofContact_Self Enquiry,TypeofContact_nan,Occupation_Free Lancer,Occupation_Large Business,Occupation_Salaried,Occupation_Small Business,Gender_Fe Male,Gender_Female,Gender_Male,...,ProductPitched_Super Deluxe,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,MaritalStatus_Unmarried,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [33]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(df_cat,1)
df.head()

,CustomerID,ProdTaken,Age,CityTier,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,...,ProductPitched_Super Deluxe,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,MaritalStatus_Unmarried,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP
0,200000,1,41.0,3,6.0,3,3.0,3.0,1.0,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,200001,0,49.0,1,14.0,3,4.0,4.0,2.0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,200002,1,37.0,1,8.0,3,4.0,3.0,7.0,1,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,200003,0,33.0,1,9.0,2,3.0,3.0,2.0,1,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,200004,0,NaN,1,8.0,2,3.0,4.0,1.0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [34]:
#How many Null Values are there per Column
df.isnull().sum(axis=0)

CustomerID                         0
ProdTaken                          0
Age                              226
CityTier                           0
DurationOfPitch                  251
NumberOfPersonVisiting             0
NumberOfFollowups                 45
PreferredPropertyStar             26
NumberOfTrips                    140
Passport                           0
PitchSatisfactionScore             0
OwnCar                             0
NumberOfChildrenVisiting          66
MonthlyIncome                    233
TypeofContact_Company Invited      0
TypeofContact_Self Enquiry         0
TypeofContact_nan                  0
Occupation_Free Lancer             0
Occupation_Large Business          0
Occupation_Salaried                0
Occupation_Small Business          0
Gender_Fe Male                     0
Gender_Female                      0
Gender_Male                        0
ProductPitched_Basic               0
ProductPitched_Deluxe              0
ProductPitched_King                0
P

## For the mock up we just drop the null Values. We will decide during the further process how these should be imputed.

In [35]:
#For evaluation of model just drop all rows with NaN - better method to be evaluated during analysis
df =df.dropna()
df.isnull().sum(axis=0)

CustomerID                       0
ProdTaken                        0
Age                              0
CityTier                         0
DurationOfPitch                  0
NumberOfPersonVisiting           0
NumberOfFollowups                0
PreferredPropertyStar            0
NumberOfTrips                    0
Passport                         0
PitchSatisfactionScore           0
OwnCar                           0
NumberOfChildrenVisiting         0
MonthlyIncome                    0
TypeofContact_Company Invited    0
TypeofContact_Self Enquiry       0
TypeofContact_nan                0
Occupation_Free Lancer           0
Occupation_Large Business        0
Occupation_Salaried              0
Occupation_Small Business        0
Gender_Fe Male                   0
Gender_Female                    0
Gender_Male                      0
ProductPitched_Basic             0
ProductPitched_Deluxe            0
ProductPitched_King              0
ProductPitched_Standard          0
ProductPitched_Super

In [36]:
# Define the features set.
X = df.copy()
X = X.drop("ProdTaken", axis=1)
X

,CustomerID,Age,CityTier,DurationOfPitch,NumberOfPersonVisiting,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,...,ProductPitched_Super Deluxe,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,MaritalStatus_Unmarried,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP
0,200000,41.0,3,6.0,3,3.0,3.0,1.0,1,2,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,200001,49.0,1,14.0,3,4.0,4.0,2.0,0,3,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,200002,37.0,1,8.0,3,4.0,3.0,7.0,1,3,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,200003,33.0,1,9.0,2,3.0,3.0,2.0,1,5,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,200005,32.0,1,8.0,3,3.0,3.0,1.0,0,5,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,204883,49.0,3,9.0,3,5.0,4.0,2.0,1,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4884,204884,28.0,1,31.0,4,5.0,3.0,3.0,1,3,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4885,204885,52.0,3,17.0,4,4.0,4.0,7.0,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4886,204886,19.0,3,16.0,3,4.0,3.0,3.0,0,5,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [37]:
# Define the target set.
y = df["ProdTaken"].values

In [38]:
#Split into Training and Testing Set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [39]:
# Splitting into Train and Test sets into an 80/20 split.
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=78, train_size=0.80)

In [40]:
# Determine the shape of our training and testing sets.
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(3302, 37)
(826, 37)
(3302,)
(826,)


In [41]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [42]:
#Test if train set scaled
print(np.mean(X_train_scaled[:,0]))
print(np.std(X_train_scaled[:,0]))

8.53867651290521e-15
0.9999999999999998


In [43]:
#Test if test set scaled
print(np.mean(X_test_scaled[:,0]))
print(np.std(X_test_scaled[:,0]))

-0.0862533763220096
1.021973313244985


## Using a basic Decision Tree model to test 

In [58]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [59]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)
predictions

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [60]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,777,71
Actual 1,43,141


In [61]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.8895348837209303

In [64]:
# Displaying Classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       0.95      0.92      0.93       848
           1       0.67      0.77      0.71       184

    accuracy                           0.89      1032
   macro avg       0.81      0.84      0.82      1032
weighted avg       0.90      0.89      0.89      1032



## Test Ensemble Learning

The concept of ensemble learning is the process of combining multiple models,to help improve the accuracy and robustness, as well as decrease variance of the model, and therefore increase the overall performance of the model.
### Random Forests
We would also want to test a Random Forest algorithm, because instead of having a single, complex tree like the ones created by decision trees, a random forest algorithm will sample the data and build several smaller, simpler decision trees. Each tree is simpler because it is built from a random subset of features.Are robust against overfitting as all of those weak learners are trained on different pieces of the data.

In [50]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [51]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [52]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [53]:
# Calculating the confusion matrix.
cm2 = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm2_df = pd.DataFrame(
    cm2, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm2_df

,Predicted 0,Predicted 1
Actual 0,837,11
Actual 1,70,114


In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [54]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,780,68
Actual 1,44,140


Accuracy Score : 0.8914728682170543
Classification Report
              precision    recall  f1-score   support

           0       0.92      0.99      0.95       848
           1       0.91      0.62      0.74       184

    accuracy                           0.92      1032
   macro avg       0.92      0.80      0.85      1032
weighted avg       0.92      0.92      0.92      1032



In [55]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.11321849, 0.11004504, 0.03111647, 0.0880044 , 0.02213373,
       0.04393958, 0.03437554, 0.05235039, 0.07621916, 0.04661059,
       0.01697435, 0.02512245, 0.1023344 , 0.01509897, 0.01468074,
       0.        , 0.00025523, 0.01605872, 0.01591133, 0.01728437,
       0.00375854, 0.01493779, 0.01551492, 0.01714878, 0.00629827,
       0.00097294, 0.00476777, 0.00253798, 0.00889279, 0.01480994,
       0.02260355, 0.01287512, 0.00242636, 0.01835755, 0.00616742,
       0.00536951, 0.00082681])

## Our mockup includes redundant columns like Customer ID, but Age and Monthly income seem to be important 

In [56]:
# Sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.11321848815800974, 'CustomerID'),
 (0.11004503806724109, 'Age'),
 (0.1023344018530924, 'MonthlyIncome'),
 (0.0880043971451117, 'DurationOfPitch'),
 (0.07621915567450435, 'Passport'),
 (0.05235039134400024, 'NumberOfTrips'),
 (0.04661058900601905, 'PitchSatisfactionScore'),
 (0.043939578295070135, 'NumberOfFollowups'),
 (0.034375537485237126, 'PreferredPropertyStar'),
 (0.031116468393998634, 'CityTier'),
 (0.0251224543997241, 'NumberOfChildrenVisiting'),
 (0.022603550455485844, 'MaritalStatus_Single'),
 (0.022133728914561817, 'NumberOfPersonVisiting'),
 (0.018357552560580137, 'Designation_Executive'),
 (0.01728436930171892, 'Occupation_Small Business'),
 (0.017148784135462523, 'ProductPitched_Basic'),
 (0.016974354879806895, 'OwnCar'),
 (0.016058722880199584, 'Occupation_Large Business'),
 (0.0159113294171342, 'Occupation_Salaried'),
 (0.015514916900018168, 'Gender_Male'),
 (0.015098967625743167, 'TypeofContact_Company Invited'),
 (0.014937793697551731, 'Gender_Female'),
 (0.0148099

# relative high accuracy, but need to check overfitting.
One of the reasons might be imbalance in the data. This would have to be addressed with over or under sampling or a combination of both.
Another reason could be noise from our unprocessed data set.